In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings(action='once')

In [3]:
import ecephys_analyses as ea
from sglxarray import load_trigger
from pyedflib import highlevel as edf
import numpy as np
from scipy.interpolate import interp1d
import xarray as xr
from datetime import datetime

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.10/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [4]:
def resample(sig, target_fs):
    f = interp1d(sig.time, sig.T, kind='cubic')
    new_times = np.arange(sig.time.min(), sig.time.max(), 1/target_fs)
    new_data = f(new_times)
    return (new_times, new_data)

In [5]:
def write_edf(sig, emg, edf_path):
    target_fs = 100.0
    sig_time_rs, sig_data_rs = resample(sig, target_fs)
    emg_time_rs, emg_data_rs = resample(emg, target_fs)
    
    signals = np.vstack([sig_data_rs, emg_data_rs])
    signal_labels = [f"LF{ch}" for ch in sig.channel.values] + ["dEMG"]
    
    signal_headers = edf.make_signal_headers(signal_labels, dimension=sig.units, sample_rate=target_fs, physical_min=signals.min(), physical_max=signals.max())
    edf.write_edf(str(edf_path), signals, signal_headers)

In [6]:
def write_alias_edfs(subject, experiment, alias, probe):
    chans = ea.get_channels(subject, experiment, probe, 'scoring')
    bin_paths = ea.get_lfp_bin_paths(subject, experiment, alias, probe=probe)
    emg_paths = ea.get_project_counterparts("SPWRs", subject, bin_paths, ".emg.nc")
    edf_paths = ea.get_project_counterparts("SPWRs", subject, bin_paths, ".edf", remove_stream=True)
    
    for bin_path, emg_path, edf_path in zip(bin_paths, emg_paths, edf_paths):
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Starting {str(bin_path)}")
        sig = load_trigger(bin_path, chans)
        emg = xr.load_dataarray(emg_path)
        write_edf(sig, emg, edf_path)
        print("Done!")

In [7]:
import matplotlib.pyplot as plt

def check_lfp_resampling(sig, time_rs, data_rs, n_seconds=4, iCh=0):
    fig, ax = plt.subplots(figsize=(36, 3))
    ax.plot(sig.time.values[:int(n_seconds*sig.fs)], sig.T.values[iCh, :int(n_seconds*sig.fs)])
    ax.plot(time_rs[:int(n_seconds*target_fs)], data_rs[iCh, :int(n_seconds*target_fs)])
    
def check_emg_resampling(emg, time_rs, data_rs, n_seconds=4, iCh=0):
    fig, ax = plt.subplots(figsize=(36, 3))
    ax.plot(emg.time.values[:int(n_seconds*emg.target_sf)], emg.values[:int(n_seconds*emg.target_sf)])
    ax.plot(time_rs[:int(n_seconds*target_fs)], data_rs[:int(n_seconds*target_fs)])

In [10]:
write_alias_edfs("CNPIX12-Santiago", "conveyor_over_water", "full_48h", "imec1")

15:21:03: Starting /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 18000039


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.10/site-packages/xarray/backends/plugins.py:102: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entrypoints = entry_points().get("xarray.backends", ())


Done!
15:24:49: Starting /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t1.imec1.lf.bin
nChan: 385, nFileSamp: 18000038
Done!
15:29:13: Starting /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t2.imec1.lf.bin
nChan: 385, nFileSamp: 18000039
Done!
15:32:45: Starting /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t3.imec1.lf.bin
nChan: 385, nFileSamp: 18000038
Done!
15:36:06: Starting /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t4.imec1.lf.bin
nChan: 385, nFileSamp: 18000039
Done!
15:39:39: Starting /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t5.imec1.lf.bin
nChan: 385, nFileSamp: 18000038
Done!
15:42:59: Starting 